In [227]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense
from tensorflow.math import confusion_matrix

In [170]:
df= pd.DataFrame(pd.read_csv("Customer.txt"))
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [171]:
for colm in df.columns:
    print(colm, df[colm].unique())

customerID ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
gender ['Female' 'Male']
SeniorCitizen [0 1]
Partner ['Yes' 'No']
Dependents ['No' 'Yes']
tenure [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
PhoneService ['No' 'Yes']
MultipleLines ['No phone service' 'No' 'Yes']
InternetService ['DSL' 'Fiber optic' 'No']
OnlineSecurity ['No' 'Yes' 'No internet service']
OnlineBackup ['Yes' 'No' 'No internet service']
DeviceProtection ['No' 'Yes' 'No internet service']
TechSupport ['No' 'Yes' 'No internet service']
StreamingTV ['No' 'Yes' 'No internet service']
StreamingMovies ['No' 'Yes' 'No internet service']
Contract ['Month-to-month' 'One year' 'Two year']
PaperlessBilling ['Yes' 'No']
PaymentMethod ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (a

In [172]:
#drop customerID
df.drop("customerID", axis=1, inplace=True)

#TotalCharges to numeric
df=df[df["TotalCharges"]!=' ']
df["TotalCharges"]=pd.to_numeric(df["TotalCharges"])

# make 0,1 for some column
df["gender"]=df["gender"].map({'Female': 0 ,'Male': 1})
df.replace(['No phone service','No internet service'],"No", inplace=True)

yes_no_col=["Partner","Dependents","PhoneService","MultipleLines","OnlineSecurity",
           "OnlineBackup","DeviceProtection","TechSupport","StreamingTV",
           "StreamingMovies","PaperlessBilling","Churn"]

for i in yes_no_col:
    df[i]=df[i].map({'Yes': 0 ,'No': 1})

for i in yes_no_col:
    df[i].replace({'Yes': 0 ,'No': 1}, inplace=True)


# get_dummies for other columns
df2=pd.get_dummies(df,["InternetService","Contract","PaymentMethod"], dtype=float)
df2.shape

#scale for some columns
scale_col=["tenure","MonthlyCharges","TotalCharges"]
scaler=MinMaxScaler()
df2[scale_col]=scaler.fit_transform(df2[scale_col])

,tenure,MonthlyCharges,TotalCharges
0,0.000000,0.115423,0.001275
1,0.464789,0.385075,0.215867
2,0.014085,0.354229,0.010310
3,0.619718,0.239303,0.210241
4,0.014085,0.521891,0.015330


In [173]:
df2["Churn"].value_counts()

Churn
1    5163
0    1869
Name: count, dtype: int64

In [228]:
#imblanced dataset (under sampling)

In [174]:
Churn_class_0=df2[df2["Churn"]==0] #size 1869
Churn_class_1=df2[df2["Churn"]==1] #size 5163
Churn_class_1_balanced=Churn_class_1.sample(Churn_class_0.shape[0])
df_balanced=pd.concat([Churn_class_0,Churn_class_1_balanced], axis=0)

In [184]:
x=df_balanced.drop('Churn',inplace=False, axis=1)
y=df_balanced['Churn']
xtrain,xtext,ytrain,ytest=train_test_split(x,y,test_size=0.25, random_state=100,)

In [192]:
model=keras.Sequential([
    keras.layers.Dense(26, input_dim=26, activation='relu'),
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid'),
    ])

model.compile(optimizer='adam',
              loss='binary_crossentropy', metrics=['accuracy'])

model.fit(xtrain, ytrain, epochs=100)

Epoch 1/100
88/88 [==============================] - 1s 2ms/step - loss: 0.6488 - accuracy: 0.6518
Epoch 2/100
88/88 [==============================] - 0s 2ms/step - loss: 0.5223 - accuracy: 0.7435
Epoch 3/100
88/88 [==============================] - 0s 2ms/step - loss: 0.5051 - accuracy: 0.7499
Epoch 4/100
88/88 [==============================] - 0s 2ms/step - loss: 0.4984 - accuracy: 0.7517
Epoch 5/100
88/88 [==============================] - 0s 2ms/step - loss: 0.4918 - accuracy: 0.7567
Epoch 6/100
88/88 [==============================] - 0s 2ms/step - loss: 0.4899 - accuracy: 0.7567
Epoch 7/100
88/88 [==============================] - 0s 2ms/step - loss: 0.4860 - accuracy: 0.7560
Epoch 8/100
88/88 [==============================] - 0s 1ms/step - loss: 0.4855 - accuracy: 0.7606
Epoch 9/100
88/88 [==============================] - 0s 1ms/step - loss: 0.4807 - accuracy: 0.7570
Epoch 10/100
88/88 [==============================] - 0s 1ms/step - loss: 0.4792 - accuracy: 0.7592
Epoch 11/

In [193]:
model.evaluate(xtext,ytest)

30/30 [==============================] - 0s 1ms/step - loss: 0.5738 - accuracy: 0.7390


[0.5738305449485779, 0.7390374541282654]

In [194]:
ypred=np.round(model.predict(xtext))

30/30 [==============================] - 0s 2ms/step


In [195]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.71      0.83      0.76       477
           1       0.78      0.64      0.71       458

    accuracy                           0.74       935
   macro avg       0.75      0.74      0.74       935
weighted avg       0.75      0.74      0.74       935



In [230]:
#imblanced dataset (over sampling)

In [229]:
Churn_class_0=df2[df2["Churn"]==0] #size 1869
Churn_class_1=df2[df2["Churn"]==1] #size 5163
Churn_class_0_balanced=Churn_class_0.sample(Churn_class_1.shape[0], replace=True)
df_balanced=pd.concat([Churn_class_0_balanced,Churn_class_1], axis=0)

In [224]:
x=df_balanced.drop('Churn',inplace=False, axis=1)
y=df_balanced['Churn']
xtrain,xtext,ytrain,ytest=train_test_split(x,y,test_size=0.25, random_state=100,stratify=y)

In [211]:
model=keras.Sequential([
    keras.layers.Dense(26, input_dim=26, activation='relu'),
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid'),
    ])

model.compile(optimizer='adam',
              loss='binary_crossentropy', metrics=['accuracy'])

model.fit(xtrain, ytrain, epochs=100)

Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5548 - accuracy: 0.7277
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4971 - accuracy: 0.7567
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4915 - accuracy: 0.7623
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4866 - accuracy: 0.7612
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4844 - accuracy: 0.7647
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4792 - accuracy: 0.7698
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4766 - accuracy: 0.7661
Epoch 8/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4738 - accuracy: 0.7723
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4715 - accuracy: 0.7729
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4689 - accura

In [212]:
model.evaluate(xtext,ytest)

81/81 [==============================] - 0s 2ms/step - loss: 0.4484 - accuracy: 0.7951


[0.44844651222229004, 0.7951200604438782]

In [213]:
ypred=np.round(model.predict(xtext))

81/81 [==============================] - 0s 2ms/step


In [214]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.75      0.88      0.81      1277
           1       0.86      0.71      0.78      1305

    accuracy                           0.80      2582
   macro avg       0.80      0.80      0.79      2582
weighted avg       0.80      0.80      0.79      2582



In [225]:
#oversampling using SMOTE
#imbalanced-learn library use pip install imbalanced-learn command
#from imblearn.over_sampling import SMOTE

#smote = SMOTE(sampling_strategy='minority')
#X_sm, y_sm = smote.fit_sample(X, y)

In [226]:
#Ensemble sampling, first train-test-split,
#make a ensemble on train data set (samle size model) and use majarity vote between models